In [ ]:
import sys
!{sys.executable} -m pip install --upgrade pip
!{sys.executable} -m pip install msgpack
!{sys.executable} -m pip install mpmath
!{sys.executable} -m pip install powerlaw
import powerlaw
import numpy as np
import scipy.io as sio

In [ ]:
np.seterr(divide='ignore', invalid='ignore')
basedir = '/Users/harangju/Developer'
x = sio.loadmat(basedir + '/durs.mat')
dur_var_name = 'durs'
(nrows,ncols) = x[dur_var_name].shape
T = x['durs_max'][0][0]

In [ ]:
# f = [[0]*ncols]*nrows
f = [0]*nrows
for i in range(0,nrows):
#     print('MLE for ({}/{},{}/{})...'.format(i+1,nrows,j+1,ncols))
    print('MLE for {}/{}...'.format(i+1,nrows))
    d = x[dur_var_name][i][j][0]
    d = d[d<T]
    if len(d) <= 1:
        continue
#     f[i] = powerlaw.Fit(d, xmin=min(d), discrete=True)
    f[i] = powerlaw.Fit(d, discrete=True)
#     for j in range(0,ncols):
#         print('MLE for ({}/{},{}/{})...'.format(i+1,nrows,j+1,ncols))
#         d = x[dur_var_name][i][j][0]
#         d = d[d<T]
#         if len(d) <= 1:
#             continue
# #         fit = powerlaw.Fit(d, xmin=min(d), discrete=True)
#         fit = powerlaw.Fit(d, discrete=True)
#         f[i][j] = fit

In [ ]:
p = np.zeros((nrows,3))
for i in range(0,nrows):
    for j in range(0,ncols):
        try:
            p[i] = [f[i].truncated_power_law.parameter1, \
                      f[i].truncated_power_law.parameter2, \
                      f[i].truncated_power_law.xmin]
        except:
            print("{} not available".format(i))
p

In [ ]:
t_pl_exp = np.zeros((nrows,2))
t_pl_tpl = np.zeros((nrows,2))
t_exp_tpl = np.zeros((nrows,2))
for i in range(0,nrows):
    try:
        R, pval = f[i].distribution_compare('power_law', 'exponential', normalized_ratio = True)
        t_pl_exp[i] = [R, pval]
        R, pval = f[i].distribution_compare('power_law', 'truncated_power_law', normalized_ratio = True)
        t_pl_tpl[i] = [R, pval]
        R, pval = f[i].distribution_compare('exponential', 'truncated_power_law', normalized_ratio = True)
        t_exp_tpl[i] = [R, pval]
    except:
        print("{} not available".format(i))

In [ ]:
sio.savemat('/Users/harangju/Developer/ft_pl.mat', {'ft_pl':p,'t_pl_exp':t_pl_exp,'t_pl_tpl':t_pl_tpl,'t_exp_tpl':t_exp_tpl})